In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import sklearn 
from keras.layers import Dense
from keras.models import Sequential
# from google.colab import drive
# drive.mount('/content/gdrive')
import os

In [2]:
df1 = pd.read_csv('/home/saichaitanya/Chaitanya/CSV files/1000 points per trajectory/May8rd10percent_RLV_data 1000 points_from86 .csv',header = None,names = ['h','v','s','omega','gamma','m','theta','Thrust','beta','time'])
df2 = pd.read_csv('/home/saichaitanya/Chaitanya/CSV files/1000 points per trajectory/May8rd10percent_RLV_data 1000 points2 .csv',header = None,names = ['h','v','s','omega','gamma','m','theta','Thrust','beta','time'])
df = pd.concat([df1,df2],ignore_index=False)
input = output =df.values
X = input[:,0:7]
y = output[:,7:9]
# Individual Data
from sklearn import preprocessing
X_norm = preprocessing.minmax_scale(X)
y_norm = preprocessing.minmax_scale(y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_norm, y_norm, test_size=0.1, random_state=42)
cross_validation_number = int(X.shape[0]*0.1)
X_val = X_train[-cross_validation_number:]
y_val = y_train[-cross_validation_number:]
X_train = X_train[:-cross_validation_number]
y_train = y_train[:-cross_validation_number]

In [3]:
def create_model2(n):
    model = Sequential()
    model.add(Dense(n,input_shape=(7,),kernel_initializer='uniform'))
    model.add(Dense(n,kernel_initializer='uniform',activation = 'relu'))
    model.add(Dense(n,kernel_initializer='uniform',activation = 'relu')) # since tanh has more nonlinearity we add it here, it also gives -ve values so , some layers which are not necessary will lead to 0 in next layer
    model.add(Dense(n,kernel_initializer='uniform',activation = 'relu'))
    model.add(Dense(n,kernel_initializer='uniform',activation = 'relu'))
    model.add(Dense(2,kernel_initializer='uniform',activation = 'sigmoid'))
    return model

In [4]:
i = 7;
model = create_model2(2**i)
opt = tf.keras.optimizers.Adam(learning_rate = 0.001)
loss = tf.keras.losses.mean_squared_error
def lr_sch3(epoch,lr):
    if (epoch >0) &(epoch<100):
      if epoch%30==0: # for every 100 epochs the learning rate varies as metioned. 
        return round(lr*np.exp(-0.45),7)
      else:
        return round(lr,7)
    elif(epoch>100):
        if epoch%20==0:
            return round(lr*0.85,9)
        else:
            return round(lr,9)
    else:
        return round(lr,9)
print("The Number of Hidden units used is: ",2**i)
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_sch3,verbose = 1)
bs = 256;
STEPS_PER_EPOCH = X_train.shape[0] / bs
save_period = 20
checkpoint_path = f"/home/saichaitanya/Chaitanya/GOOGLE COLAB  DOCS/10 percent variation/1000 points 7 states and 2 op with val_set having 128 neurons and batchsize 256/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_weights_only=True,
                                                verbose=1,
                                            save_freq=int(save_period*STEPS_PER_EPOCH))
model.compile(optimizer = opt, loss = loss, metrics = 'accuracy')
iter1 = 150

The Number of Hidden units used is:  128


2022-05-13 19:44:15.933323: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-05-13 19:44:15.933379: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-05-13 19:44:15.934399: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


hist = model.fit(X_train,y_train,epochs = iter1,batch_size = bs,shuffle = True,use_multiprocessing = True,callbacks=[lr_scheduler,cp_callback],validation_data=(X_val, y_val))

In [5]:
model.load_weights(f"/home/saichaitanya/Chaitanya/GOOGLE COLAB  DOCS/10 percent variation/1000 points 7 states and 2 op with val_set having 128 neurons and batchsize 256/cp.ckpt")

model.save(f"/home/saichaitanya/Chaitanya/GOOGLE COLAB  DOCS/10 percent variation/1000 points 7 states and 2 op with val_set having 128 neurons and batchsize 256/cp.ckpt")

In [6]:
model.predict(X_test)

array([[0.05342954, 0.6702719 ],
       [0.05446598, 0.5860089 ],
       [0.5321479 , 0.49459007],
       ...,
       [0.05198124, 0.6522221 ],
       [0.2029373 , 0.62137926],
       [0.31623423, 0.47534943]], dtype=float32)

In [7]:
y_test

array([[0.05210304, 0.67865059],
       [0.05867656, 0.61604185],
       [0.55106001, 0.49590121],
       ...,
       [0.05159365, 0.65077814],
       [0.20756319, 0.62336328],
       [0.31121137, 0.48170023]])

In [8]:
model.evaluate(X_test,y_test)

6848/6848 [==============================] - 13s 2ms/step - loss: 2.0550e-04 - accuracy: 0.9904


[0.00020550069166347384, 0.9904481172561646]

In [9]:
model.evaluate(X_train,y_train)

54780/54780 [==============================] - 102s 2ms/step - loss: 1.9943e-04 - accuracy: 0.9909


[0.0001994267076952383, 0.9908913373947144]

In [12]:
lr = 0.000187344
opt = tf.keras.optimizers.Adam(learning_rate = lr)
loss = tf.keras.losses.mean_squared_error
iter2 = 150
def lr_sch(epoch,lr):
    if (epoch >0) &(epoch<25):
      if epoch%5==0: # for every 100 epochs the learning rate varies as metioned. 
        return round(lr*np.exp(-0.45),7)
      else:
        return round(lr,7)
    elif(epoch>25):
        if epoch%5==0:
            return round(lr*0.85,9)
        else:
            return round(lr,9)
    else:
        return round(lr,9)
print("The Number of Hidden units used is: ",2**i)
lr_scheduler2 = tf.keras.callbacks.LearningRateScheduler(lr_sch,verbose = 1)
checkpoint_path = f"/home/saichaitanya/Chaitanya/GOOGLE COLAB  DOCS/10 percent variation/1000 points 7 states and 2 op with val_set having 128 neurons and batchsize 256/cp.ckpt"
bs2 = 1024
STEPS_PER_EPOCH = X_train.shape[0] / bs
save_period = 20
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_weights_only=True,
                                                verbose=1,
                                            save_freq=int(save_period*STEPS_PER_EPOCH))
model.compile(optimizer = opt, loss = loss, metrics = 'accuracy')

The Number of Hidden units used is:  128


In [13]:
hist = model.fit(X_train,y_train,epochs = iter2,batch_size = bs2,shuffle = True,use_multiprocessing = True,callbacks=[lr_scheduler2,cp_callback],validation_data=(X_val, y_val))


Epoch 1: LearningRateScheduler setting learning rate to 0.000187344.
Epoch 1/150
1712/1712 [==============================] - 13s 7ms/step - loss: 2.1427e-04 - accuracy: 0.9902 - val_loss: 2.0843e-04 - val_accuracy: 0.9899 - lr: 1.8734e-04

Epoch 2: LearningRateScheduler setting learning rate to 0.0001873.
Epoch 2/150
1712/1712 [==============================] - 12s 7ms/step - loss: 2.1413e-04 - accuracy: 0.9902 - val_loss: 2.7373e-04 - val_accuracy: 0.9853 - lr: 1.8730e-04

Epoch 3: LearningRateScheduler setting learning rate to 0.0001873.
Epoch 3/150
1712/1712 [==============================] - 12s 7ms/step - loss: 2.1483e-04 - accuracy: 0.9902 - val_loss: 2.1713e-04 - val_accuracy: 0.9910 - lr: 1.8730e-04

Epoch 4: LearningRateScheduler setting learning rate to 0.0001873.
Epoch 4/150
1712/1712 [==============================] - 12s 7ms/step - loss: 2.1197e-04 - accuracy: 0.9904 - val_loss: 1.9758e-04 - val_accuracy: 0.9915 - lr: 1.8730e-04

Epoch 5: LearningRateScheduler setting le

Epoch 35/150
1712/1712 [==============================] - 12s 7ms/step - loss: 1.6911e-04 - accuracy: 0.9923 - val_loss: 1.7338e-04 - val_accuracy: 0.9925 - lr: 2.6265e-05

Epoch 36: LearningRateScheduler setting learning rate to 2.2325e-05.
Epoch 36/150
1712/1712 [==============================] - 12s 7ms/step - loss: 1.6812e-04 - accuracy: 0.9924 - val_loss: 1.6696e-04 - val_accuracy: 0.9927 - lr: 2.2325e-05

Epoch 37: LearningRateScheduler setting learning rate to 2.2325e-05.
Epoch 37/150
1712/1712 [==============================] - 12s 7ms/step - loss: 1.6881e-04 - accuracy: 0.9923 - val_loss: 1.6699e-04 - val_accuracy: 0.9923 - lr: 2.2325e-05

Epoch 38: LearningRateScheduler setting learning rate to 2.2325e-05.
Epoch 38/150
1712/1712 [==============================] - 12s 7ms/step - loss: 1.6733e-04 - accuracy: 0.9925 - val_loss: 1.6848e-04 - val_accuracy: 0.9923 - lr: 2.2325e-05

Epoch 39: LearningRateScheduler setting learning rate to 2.2325e-05.
Epoch 39/150
1712/1712 [========

Epoch 69/150
1712/1712 [==============================] - 12s 7ms/step - loss: 1.6134e-04 - accuracy: 0.9927 - val_loss: 1.6200e-04 - val_accuracy: 0.9930 - lr: 8.4190e-06

Epoch 70: LearningRateScheduler setting learning rate to 8.419e-06.
Epoch 70/150
1712/1712 [==============================] - 12s 7ms/step - loss: 1.6163e-04 - accuracy: 0.9927 - val_loss: 1.6733e-04 - val_accuracy: 0.9922 - lr: 8.4190e-06

Epoch 71: LearningRateScheduler setting learning rate to 7.156e-06.
Epoch 71/150
1712/1712 [==============================] - 12s 7ms/step - loss: 1.6125e-04 - accuracy: 0.9927 - val_loss: 1.6254e-04 - val_accuracy: 0.9929 - lr: 7.1560e-06

Epoch 72: LearningRateScheduler setting learning rate to 7.156e-06.
Epoch 72/150
1712/1712 [==============================] - 12s 7ms/step - loss: 1.6095e-04 - accuracy: 0.9928 - val_loss: 1.6181e-04 - val_accuracy: 0.9929 - lr: 7.1560e-06

Epoch 73: LearningRateScheduler setting learning rate to 7.156e-06.
Epoch 73/150
1712/1712 [============

Epoch 102/150
1712/1712 [==============================] - 12s 7ms/step - loss: 1.5907e-04 - accuracy: 0.9928 - val_loss: 1.6055e-04 - val_accuracy: 0.9930 - lr: 2.7000e-06

Epoch 103: LearningRateScheduler setting learning rate to 2.7e-06.
Epoch 103/150
1712/1712 [==============================] - 12s 7ms/step - loss: 1.5904e-04 - accuracy: 0.9928 - val_loss: 1.6075e-04 - val_accuracy: 0.9929 - lr: 2.7000e-06

Epoch 104: LearningRateScheduler setting learning rate to 2.7e-06.
Epoch 104/150
1712/1712 [==============================] - 12s 7ms/step - loss: 1.5900e-04 - accuracy: 0.9928 - val_loss: 1.6130e-04 - val_accuracy: 0.9929 - lr: 2.7000e-06

Epoch 105: LearningRateScheduler setting learning rate to 2.7e-06.
Epoch 105/150
1712/1712 [==============================] - 12s 7ms/step - loss: 1.5896e-04 - accuracy: 0.9928 - val_loss: 1.6061e-04 - val_accuracy: 0.9929 - lr: 2.7000e-06

Epoch 106: LearningRateScheduler setting learning rate to 2.295e-06.
Epoch 106/150
1712/1712 [=========

Epoch 136/150
1712/1712 [==============================] - 12s 7ms/step - loss: 1.5810e-04 - accuracy: 0.9929 - val_loss: 1.6083e-04 - val_accuracy: 0.9928 - lr: 8.6500e-07

Epoch 137: LearningRateScheduler setting learning rate to 8.65e-07.
Epoch 137/150
1712/1712 [==============================] - 12s 7ms/step - loss: 1.5813e-04 - accuracy: 0.9929 - val_loss: 1.6035e-04 - val_accuracy: 0.9929 - lr: 8.6500e-07

Epoch 138: LearningRateScheduler setting learning rate to 8.65e-07.
Epoch 138/150
1712/1712 [==============================] - 12s 7ms/step - loss: 1.5812e-04 - accuracy: 0.9929 - val_loss: 1.6025e-04 - val_accuracy: 0.9929 - lr: 8.6500e-07

Epoch 139: LearningRateScheduler setting learning rate to 8.65e-07.
Epoch 139/150
1712/1712 [==============================] - 12s 7ms/step - loss: 1.5811e-04 - accuracy: 0.9929 - val_loss: 1.6074e-04 - val_accuracy: 0.9930 - lr: 8.6500e-07

Epoch 140: LearningRateScheduler setting learning rate to 8.65e-07.
Epoch 140/150
1712/1712 [=======

In [14]:
model.save(f"/home/saichaitanya/Chaitanya/GOOGLE COLAB  DOCS/10 percent variation/1000 points 7 states and 2 op with val_set having 128 neurons and batchsize 256/cp.ckpt")

2022-05-13 22:15:41.208157: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /home/saichaitanya/Chaitanya/GOOGLE COLAB  DOCS/10 percent variation/1000 points 7 states and 2 op with val_set having 128 neurons and batchsize 256/cp.ckpt/assets
